## The Kafka Connect API

In this exercise we're going to make use of the Kafka Connect API.

[See the documentation for more information on any of these actions](https://docs.confluent.io/current/connect/references/restapi.html).

### Viewing Connectors

First, we can view connector-plugins:

`curl http://localhost:8083/connector-plugins | python -m json.tool`

Quick note, the `| python -m json.tool` above simply takes the output of the `curl` command and
prints the JSON nicely. You can omit this if you'd like!

### Create a Connector

Lets create a connector. We'll dive into more details on how this works later.

```
curl -X POST -H 'Content-Type: application/json' -d '{
    "name": "first-connector",
    "config": {
        "connector.class": "FileStreamSource",
        "tasks.max": 1,
        "file": "/var/log/journal/confluent-kafka-connect.service.log",
        "topic": "kafka-connect-logs"
    }
  }' \
  http://localhost:8083/connectors
```

### List connectors

We can list all configured connectors with:

`curl http://localhost:8083/connectors | python -m json.tool`

You can see our connector in the list.

### Detailing connectors

Let's list details on our connector:

`curl http://localhost:8083/connectors/first-connector | python -m json.tool`

### Status

Let's inspect the status of our connector:

`curl http://localhost:8083/connectors/first-connector/status | python -m json.tool`

Note to myself: There is an error with the task in the connector. Something in the
configuration seems to be off:

```
root@988961615cc1:/home/workspace# curl http://localhost:8083/connectors/first-connector/status | python -m json.tool
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current/f
                                 Dload  Upload   Total   Spent    Left  Speed
100  1602  100  1602    0     0  48545      0 --:--:-- --:--:-- --:--:-- 50062
{
    "name": "first-connector",
    "connector": {
        "state": "RUNNING",
        "worker_id": "172.18.0.2:8083"
    },
    "tasks": [
        {
            "id": 0,
            "state": "FAILED",
            "worker_id": "172.18.0.2:8083",
            "trace": "io.confluent.common.config.ConfigException: Missing required configuration \"schema.registry.url\" which has no default value.\n\tat io.confluent.common.config.ConfigDef.parse(ConfigDef.java:251)\n\tat io.confluent.common.config.AbstractConfig.<init>(AbstractConfig.java:78)\n\tat io.confluent.kafka.serializers.AbstractKafkaAvroSerDeConfig.<init>(AbstractKafkaAvroSerDeConfig.java:109)\n\tat io.confluent.connect.avro.AvroConverterConfig.<init>(AvroConverterConfig.java:27)\n\tat io.confluent.connect.avro.AvroConverter.configure(AvroConverter.java:61)\n\tat org.apache.kafka.connect.runtime.isolation.Plugins.newConverter(Plugins.java:291)\n\tat org.apache.kafka.connect.runtime.Worker.startTask(Worker.java:445)\n\tat org.apache.kafka.connect.runtime.distributed.DistributedHerder.startTask(DistributedHerder.java:865)\n\tat org.apache.kafka.connect.runtime.distributed.DistributedHerder.access$1600(DistributedHerder.java:110)\n\tat org.apache.kafka.connect.runtime.distributed.DistributedHerder$13.call(DistributedHerder.java:880)\n\tat org.apache.kafka.connect.runtime.distributed.DistributedHerder$13.call(DistributedHerder.java:876)\n\tat java.util.concurrent.FutureTask.run(FutureTask.java:266)\n\tat java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)\n\tat java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)\n\tat java.lang.Thread.run(Thread.java:748)\n"
        }
    ],
    "type": "source"
}
```

Adding the missing requirement to the config does not solve the issue. The command below
yields the same failed task

```
curl -X POST -H 'Content-Type: application/json' -d '{
    "name": "first-connector",
    "config": {
        "connector.class": "FileStreamSource",
        "tasks.max": 1,
        "file": "/var/log/journal/confluent-kafka-connect.service.log",
        "topic": "kafka-connect-logs",
        "schema.registry.url": "http://localhost:8081"
    }
  }' \
  http://localhost:8083/connectors
```

### Pausing connectors

Sometimes its desirable to pause or restart connectors:

To pause:

`curl -X PUT http://localhost:8083/connectors/first-connector/pause`

To restart:

`curl -X POST http://localhost:8083/connectors/first-connector/restart`

Note to myself: The restart actually does not work! To resume the paused connector, use

`curl -X PUT http://localhost:8083/connectors/first-connector/resume`

### Deleting connectors

Finally, to delete your connector:

`curl -X DELETE http://localhost:8083/connectors/first-connector`
